<a href="https://colab.research.google.com/github/suridian/Applied-Deep-Learning-Homework-3/blob/main/LearnMLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instaling MLFlow
!pip install mlflow

In [ ]:
#import core modules
import os
import warnings
import sys

In [ ]:
#import third party modules
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging

In [ ]:
#initializing logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [ ]:
#function - evaluation matrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
warnings.filterwarnings("ignore")
np.random.seed(40)

In [ ]:
# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

In [ ]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [ ]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [ ]:
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
# l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.5



# alpha = sys.argv[1] if len(sys.argv) > 1 else 0.5
# l1_ratio = sys.argv[2] if len(sys.argv) > 2 else 0.5

alpha = 0
l1_ratio = 0
if len(sys.argv) > 1:
  alpha = 0.5
if len(sys.argv) > 2:
  l1_ratio = 0.5

['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-eb8f3fce-3837-4aa5-9a5a-1ee30339aac5.json']
0.5


In [ ]:
with mlflow.start_run():
  lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
  lr.fit(train_x, train_y)

  predicted_qualities = lr.predict(test_x)

  (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

  print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
  print("  RMSE: %s" % rmse)
  print("  MAE: %s" % mae)
  print("  R2: %s" % r2)

  mlflow.log_param("alpha", alpha)
  mlflow.log_param("l1_ratio", l1_ratio)
  mlflow.log_metric("rmse", rmse)
  mlflow.log_metric("r2", r2)
  mlflow.log_metric("mae", mae)

  tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [ ]:
# Model registry does not work with file store
if tracking_url_type_store != "file":

  # Register the model
  # There are other ways to use the Model Registry, which depends on the use case,
  # please refer to the doc for more information:
  # https://mlflow.org/docs/latest/model-registry.html#api-workflow
  mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
else:
  mlflow.sklearn.log_model(lr, "model")